In [1]:
from dataset import SiameseGoogleFer

In [4]:
import time
t = time.time()
train_path = "data/faceexp-comparison-data-train-public.csv"
test_path = "data/faceexp-comparison-data-test-public.csv"
train_dataset = SiameseGoogleFer(train_path, train_flag=True)
print(time.time()-t)
#test_dataset = SiameseGoogleFer(test_path, train_flag=False)

1
2
3
4
5
6
7
8
<Response [410]>
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
<Response [404]>
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
77.3010880947113


In [5]:
test_dataset = SiameseGoogleFer(test_path, train_flag=False)
train_dataset.__len__()

1
2
3
4
<Response [404]>
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
<Response [404]>
31
32
33
34
<Response [404]>
35
36
37
38
<Response [410]>
39
<Response [410]>
40
<Response [410]>
41
<Response [410]>
42
<Response [410]>
43
<Response [410]>
44
<Response [410]>
45
<Response [410]>
46
<Response [410]>
47
<Response [410]>
48
<Response [410]>
49
<Response [410]>
50
<Response [410]>
51
<Response [410]>
52
<Response [410]>
53
<Response [410]>
54
<Response [410]>
55
<Response [410]>
56
<Response [410]>
57
<Response [410]>
58
<Response [410]>
59
60
61
62
63
64
65
<Response [410]>
66
67
68
69
<Response [404]>
70
71
72
73
74
75
<Response [410]>
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
<Response [404]>
97
98
99
100


30

In [1]:
from dataset import *

In [8]:
run train.py

<Response [410]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [404]>
<Response [410]>
<Response [404]>
  + Number of params: 447340
Train Epoch: 1 [0/30]	Loss: 0.1343 (0.1343) 	Acc: 100.00% (100.00%) 	Emb_Norm: 0.49 (0.49)
Train Epoch: 1 [20/30]	Loss: 0.6547 (3.7177) 	Acc: 0.00% (47.62%) 	Emb_Norm: 0.93 (6.31)

Test set: Average loss: 0.2898, Accuracy: 41.67%

Train Epoch: 2 [0/30]	Loss: 0.0000 (0.0000) 	Acc: 100.00% (100.00%) 	Emb_Norm: 2.79 (2.79)
Train Epoch: 2 [20/30]	Loss: 0.2265 (1.1706) 	Acc: 0.00% (47.62%) 	Emb_Norm: 0.28 (1.59)

Test set: Average loss: 0.1995, Accuracy: 41.67%


In [3]:
acc

NameError: name 'acc' is not defined

In [ ]:
x = 